# Pinecone DB (프리티어 사용)

In [1]:
!pip install pinecone

In [ ]:
# Pinecone index 생성 (웹에서도 생성 가능)
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_6ASWpC_8HCC6S47t6TTJT57ffj5sjbrKEGB96zYxYAAYPEAeEDUrZvmCuS4maPB92AxmbD")
index_name = "reviewtest"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-bdry0d8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
reviews = [
    {"id": "1", "text": "이 노트북 정말 좋아요! 속도도 빠르고 디자인도 마음에 들어요.", "sentiment": "positive"},
    {"id": "2", "text": "디스플레이가 너무 어두워요. 화면이 선명하지 않아서 불편합니다.", "sentiment": "negative"},
    {"id": "3", "text": "배터리가 오래가고, 가벼워서 휴대하기 좋아요.", "sentiment": "positive"},
    {"id": "4", "text": "노트북이 자주 끊겨서 작업하기 불편해요. 비추천합니다.", "sentiment": "negative"},
    {"id": "5", "text": "이 노트북은 성능이 뛰어나고 게임도 잘 돌아가요. 매우 만족합니다.", "sentiment": "positive"},
    {"id": "6", "text": "키보드가 불편하고, 손목이 아파요. 다음에는 다른 제품을 선택할 것 같아요.", "sentiment": "negative"},
    {"id": "7", "text": "가성비 최고! 가격에 비해 성능이 훌륭해요.", "sentiment": "positive"},
    {"id": "8", "text": "정말 실망이에요. 화면이 자주 깜빡거려서 사용하기 힘들어요.", "sentiment": "negative"},
    {"id": "9", "text": "디자인이 예쁘고, 성능도 괜찮아요. 오랜 시간 사용해도 발열이 적어요.", "sentiment": "positive"},
    {"id": "10", "text": "구매하고 나서 몇 달 안 가서 고장이 나버렸어요. 수리비가 비쌉니다.", "sentiment": "negative"},
    {"id": "11", "text": "어떤 작업이든 다 잘 처리되네요. 게임이나 영상 편집도 무리 없이 가능해요.", "sentiment": "positive"},
    {"id": "12", "text": "가격에 비해 성능이 너무 부족해요. 작업할 때 자주 느려져서 짜증이 나요.", "sentiment": "negative"},
    {"id": "13", "text": "이 노트북은 디자인도 멋지고, 속도도 빠릅니다. 정말 만족스러워요!", "sentiment": "positive"},
    {"id": "14", "text": "사용 중 너무 뜨겁고, 소음도 심해요. 절대 추천하지 않습니다.", "sentiment": "negative"},
    {"id": "15", "text": "경량이라 이동할 때 매우 편리하고, 배터리도 오래가요. 좋은 선택이었어요.", "sentiment": "positive"},
    {"id": "16", "text": "불량품이 왔어요. 화면에 이상이 생겨서 교환 요청했지만 번거로웠어요.", "sentiment": "negative"},
    {"id": "17", "text": "좋은 제품입니다! 화면이 선명하고, 멀티태스킹도 잘 되네요.", "sentiment": "positive"},
    {"id": "18", "text": "구매 후 사용 중 멈춤 현상이 자주 발생하고, 해결이 안 돼요.", "sentiment": "negative"},
    {"id": "19", "text": "너무 빠르고 효율적이에요! 업무용으로 딱 좋은 제품입니다.", "sentiment": "positive"},
    {"id": "20", "text": "키보드가 너무 불편하고, 자주 오작동이 일어나요. 실망했습니다.", "sentiment": "negative"}
]

In [ ]:
# Pinecone index 및 임베딩 모델 로드
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key="pcsk_6ASWpC_8HCC6S47t6TTJT57ffj5sjbrKEGB96zYxYAAYPEAeEDUrZvmCuS4maPB92AxmbD")
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# index에 리뷰 데이터 추가
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([
        (
            review['id'], 
            review_embed,
            {
                'text': review['text'], 
                'sentiment': review['sentiment']
            }
        )
    ])

### 리뷰 데이터 조회

In [12]:
query_text = "이 노트북의 성능은 괜찮은가요?"
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

results

{'matches': [{'id': '13',
              'metadata': {'sentiment': 'positive',
                           'text': '이 노트북은 디자인도 멋지고, 속도도 빠릅니다. 정말 만족스러워요!'},
              'score': 0.753723145,
              'values': []},
             {'id': '9',
              'metadata': {'sentiment': 'positive',
                           'text': '디자인이 예쁘고, 성능도 괜찮아요. 오랜 시간 사용해도 발열이 적어요.'},
              'score': 0.725854218,
              'values': []},
             {'id': '14',
              'metadata': {'sentiment': 'negative',
                           'text': '사용 중 너무 뜨겁고, 소음도 심해요. 절대 추천하지 않습니다.'},
              'score': 0.71279937,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [13]:
for review in results['matches']:
    metadata = review['metadata']
    print(metadata['text'], f'(감성: {metadata["sentiment"]})')

이 노트북은 디자인도 멋지고, 속도도 빠릅니다. 정말 만족스러워요! (감성: positive)
디자인이 예쁘고, 성능도 괜찮아요. 오랜 시간 사용해도 발열이 적어요. (감성: positive)
사용 중 너무 뜨겁고, 소음도 심해요. 절대 추천하지 않습니다. (감성: negative)


### 리뷰 감성 분석

In [14]:
query_text = "배터리도 빨리 닳고 벽돌보다 무거워요. 비추천."
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True
)

results

{'matches': [{'id': '3',
              'metadata': {'sentiment': 'positive',
                           'text': '배터리가 오래가고, 가벼워서 휴대하기 좋아요.'},
              'score': 0.848484695,
              'values': []},
             {'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '디스플레이가 너무 어두워요. 화면이 선명하지 않아서 불편합니다.'},
              'score': 0.822854757,
              'values': []},
             {'id': '13',
              'metadata': {'sentiment': 'positive',
                           'text': '이 노트북은 디자인도 멋지고, 속도도 빠릅니다. 정말 만족스러워요!'},
              'score': 0.806697249,
              'values': []},
             {'id': '15',
              'metadata': {'sentiment': 'positive',
                           'text': '경량이라 이동할 때 매우 편리하고, 배터리도 오래가요. 좋은 선택이었어요.'},
              'score': 0.794718564,
              'values': []},
             {'id': '1',
              'metadata': {'sentiment': 'positive',
                           'text': '이 노트북 정말 좋아요! 속도도 빠

In [ ]:
sentiment_counts = {
    "positive": 0,
    "negative": 0
}

for review in results['matches']:
    sentiment = review['metadata']["sentiment"]
    sentiment_counts[sentiment] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: positive
